# Libraries

In [47]:
import os
import pandas as pd
import warnings
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
warnings.filterwarnings("ignore")

# Data Preprocessing

In [60]:
def addGrandTotal(df):
#     df['Grand Total'] = df.loc[:, df.columns != 'Row Labels'].sum(axis=1)
    grand_total = df.sum()
    grand_total_row = pd.DataFrame([grand_total], columns=grand_total.index)
    grand_total_row['Row Labels'] = 'Grand Total'
    df_with_grand_total = pd.concat([df, grand_total_row], ignore_index=True)
    return df_with_grand_total

def renameColumns(df):
    new_column_names = []
    for col in df.columns:
        if len(str(col)) == 6 and str(col).isdigit():
            year = str(col)[:4]
            week = str(col)[4:]
            new_column_name = f'{year}-{week}'
            new_column_names.append(new_column_name)
        else:
            new_column_names.append(col)
    df.columns = new_column_names  
    return df

def limit_weeks(df):
    week_columns = [col for col in df.columns if col not in ['Row Labels', 'Grand Total']]
    sorted_week_columns = sorted(week_columns)
    limited_columns = sorted_week_columns[-49:]  # Adjust if fewer columns are needed   
    columns_to_keep = ['Row Labels'] + limited_columns + ['Grand Total'] 
    valid_columns = [col for col in columns_to_keep if col in df.columns]   
    filtered_df = df[valid_columns]    
    return filtered_df


def process_excel(path):
    df = pd.read_excel(path)
    row_labels_index = df[df.eq("Row Labels").any(axis=1)].index[0]
    df.columns = df.iloc[row_labels_index]
    df = df.iloc[row_labels_index + 1:, :].reset_index(drop=True)
    df = df.dropna(how='all', axis=0).dropna(how='all', axis=1)
    df['Row Labels'] = df['Row Labels'].replace([0, "0"], "LineX1") # as there is one row with 0 label in some files
    df = df.fillna(0)
    df.columns = [col.replace('Sum of ', '') for col in df.columns]
    df['Row Labels'] = df['Row Labels'].replace(["...", "'"], "LineX1") # as there is one row with 0 label in some files
    df['Row Labels'] = df['Row Labels'].replace([0, "0"], "LineX2") # as there is one row with Nan (so doing it after filling Nans) in some files
    df = df[~df['Row Labels'].str.contains(r'X', regex=True)]
    df = df[~df['Row Labels'].str.contains(r'^CW', regex=True)]
    df = df[~df['Row Labels'].str.contains(r'blank', regex=True)]
    week_columns = [col for col in df.columns if col != 'Row Labels']
    sorted_week_columns = sorted(week_columns, key=lambda x: int(x))
    df = df[['Row Labels'] + sorted_week_columns]
    df['Grand Total'] = df.loc[:, df.columns != 'Row Labels'].sum(axis=1)
    if len(df.index) == 9 and "Grand Total" not in df['Row Labels']:
        df = addGrandTotal(df)
    df = renameColumns(df)
    df = limit_weeks(df)
    return df

directory = 'E:/Fiverr/1st/Data/Demand update every week/'
df_list = []
files = os.listdir(directory)
rowColInfo = []
for file in files:
    file_path = os.path.join(directory, file)
    df = process_excel(file_path)
    rowColInfo.append([len(df.index),len(df.columns)])

for i in range(len(rowColInfo)):
    print("File Name: ", files[i])
    print("rows: " ,rowColInfo[i][0], "cols: ", rowColInfo[i][1])

File Name:  CW0112023.xlsx
rows:  10 cols:  51
File Name:  CW0122023.xlsx
rows:  10 cols:  51
File Name:  CW0132023.xlsx
rows:  10 cols:  51
File Name:  CW0142023.xlsx
rows:  10 cols:  51
File Name:  CW0152023.xlsx
rows:  10 cols:  51
File Name:  CW0162023.xlsx
rows:  10 cols:  51
File Name:  CW0172023.xlsx
rows:  10 cols:  51
File Name:  CW0182030.xlsx
rows:  10 cols:  51
File Name:  CW0192023.xlsx
rows:  10 cols:  51
File Name:  CW0202023.xlsx
rows:  10 cols:  51
File Name:  CW0212023.xlsx
rows:  10 cols:  51
File Name:  CW022023.xlsx
rows:  10 cols:  51
File Name:  CW0222023.xlsx
rows:  10 cols:  51
File Name:  CW0232023.xlsx
rows:  10 cols:  51
File Name:  CW0242023.xlsx
rows:  10 cols:  51
File Name:  CW0252023.xlsx
rows:  10 cols:  51
File Name:  CW0262023.xlsx
rows:  10 cols:  51
File Name:  CW0272023.xlsx
rows:  10 cols:  51
File Name:  CW0282023.xlsx
rows:  10 cols:  51
File Name:  CW0292023.xlsx
rows:  10 cols:  51
File Name:  CW0302023.xlsx
rows:  10 cols:  51
File Name:  CW

In [62]:
directory = 'E:/Fiverr/1st/Data/Demand update every week/'
merged_df_demand = pd.DataFrame()
for i, file in enumerate(files):
    file_path = os.path.join(directory, file)
    df = process_excel(file_path)
    
    if i == 0:
        merged_df_demand = df
    else:
        new_columns = [col for col in df.columns if col not in merged_df_demand.columns and col != 'Row Labels']
        df_new_weeks = df[['Row Labels'] + new_columns]
        merged_df_demand = pd.merge(merged_df_demand, df_new_weeks, on='Row Labels', how='outer')

merged_df_demand

,Row Labels,2023-15,2023-16,2023-17,2023-18,2023-19,2023-20,2023-21,2023-22,2023-23,...,2024-49,2024-44,2024-47,2024-48,2024-50,2024-51,2025-02,2025-03,2025032,2025033
0,Grand Total,121323,122165,91073.833333,99948.833333,97093.333333,93476,92689,93298,94859,...,852,10350,2040,1500,4130,7200,1980,6450,9946,7818
1,Line 1,27545,17485,17342.500000,17342.500000,20550.000000,16250,17175,19300,18025,...,0,0,0,0,0,0,0,0,0,2860
2,Line 1-2,1738,1496,2449.333333,2449.333333,2449.333333,2486,2838,2508,1452,...,0,0,0,0,0,0,0,0,110,0
3,Line 2,23556,25663,15189.000000,15189.000000,16378.000000,18754,17276,18224,17180,...,0,2550,0,0,0,0,0,0,2010,0
4,Line 3,23771,25208,18592.000000,17467.000000,16911.000000,18945,16195,19471,19399,...,0,0,0,0,0,0,0,0,0,0
5,Line 4,13214,15074,12174.500000,12174.500000,6231.000000,9724,10060,8943,10702,...,852,1780,0,0,1600,0,1980,2460,2460,1490
6,Line 5,9376,10182,8285.500000,8285.500000,5454.000000,9316,9485,7796,10976,...,0,2010,0,0,2420,0,0,740,960,1014
7,Line 6,8033,7341,6117.500000,6117.500000,6182.000000,6573,6726,8520,7771,...,0,422,0,0,22,0,0,274,0,200
8,Line 7,1530,1710,1530.000000,1530.000000,1530.000000,360,1890,1890,1890,...,0,1788,0,0,0,0,0,710,710,0
9,SGB,12560,18006,9393.500000,19393.500000,20704.000000,11068,10672,6206,6936,...,0,1800,2040,1500,66,7200,0,2266,3696,2232


# Tried to train LSTM but didn't work due to lack of data

In [63]:
# def prepare_lstm_data(df):
#     scaler = MinMaxScaler()
#     df_scaled = df.copy()
    
#     # Normalize demand columns
#     df_scaled[week_columns] = scaler.fit_transform(df[week_columns])
    
#     # Drop non-numeric columns for LSTM input
#     df_scaled = df_scaled.drop(['Row Labels'], axis=1)
    
#     # Reshape data for LSTM
#     # Assuming a sliding window approach
#     def create_sequences(data, seq_length):
#         xs, ys = [], []
#         for i in range(len(data) - seq_length):
#             x = data[i:i+seq_length]
#             y = data[i+seq_length]
#             xs.append(x)
#             ys.append(y)
#         return np.array(xs), np.array(ys)

#     seq_length = 52  # Number of weeks to consider for each sequence
#     X, y = create_sequences(df_scaled.values, seq_length)
    
#     return X, y

# X, y = prepare_lstm_data(combined_df)
# y

In [64]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [65]:
# def build_lstm_model(input_shape):
#     model = Sequential()
#     model.add(LSTM(50, return_sequences=True, input_shape=input_shape))
#     model.add(LSTM(50, return_sequences=False))
#     model.add(Dense(1))  # Output layer
    
#     model.compile(optimizer='adam', loss='mean_squared_error')
#     return model

# input_shape = (X_train.shape[1], X_train.shape[2])
# model = build_lstm_model(input_shape)
# model.summary()

In [66]:
# history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.1)

In [67]:
# y_pred = model.predict(X_test)

In [69]:
# test_loss = model.evaluate(X_test, y_test)
# print(f'Test Loss: {test_loss}')

In [70]:
# # Plot training & validation loss values
# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('Model Loss')
# plt.xlabel('Epoch')
# plt.ylabel('Loss')
# plt.legend(['Train', 'Validation'], loc='upper right')
# plt.show()

# # Plot predictions vs true values
# plt.plot(y_test, label='True Values')
# plt.plot(y_pred, label='Predictions')
# plt.title('True Values vs Predictions')
# plt.xlabel('Time')
# plt.ylabel('Demand')
# plt.legend()
# plt.show()


# Final Version

# Dataset Prepration

In [16]:
import os
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

def preprocess_sales_data(path):
    def preprocess(df):
        df = df.drop(columns=['Grand Total'], errors='ignore')
        df = df[df['Row Labels'] != 'Grand Total']
        df = df.fillna(0)
        df = df[1:].reset_index(drop=True)
        return df

    merged_df = pd.DataFrame()
    
    for file in os.listdir(path):
        if file.endswith(".csv"):
            file_path = os.path.join(path, file)
            df = pd.read_csv(file_path)
            df = preprocess(df)
            if merged_df.empty:
                merged_df = df
            else:
                merged_df = pd.merge(merged_df, df, on='Row Labels', how='outer')

    return merged_df

def preprocess_demand_data(path):
    
    def addGrandTotal(df):
        grand_total = df.sum()
        grand_total_row = pd.DataFrame([grand_total], columns=grand_total.index)
        grand_total_row['Row Labels'] = 'Grand Total'
        df_with_grand_total = pd.concat([df, grand_total_row], ignore_index=True)
        return df_with_grand_total

    def renameColumns(df):
        new_column_names = []
        for col in df.columns:
            if len(str(col)) == 6 and str(col).isdigit():
                year = str(col)[:4]
                week = str(col)[4:]
                new_column_name = f'{year}-{week}'
                new_column_names.append(new_column_name)
            else:
                new_column_names.append(col)
        df.columns = new_column_names  
        return df

    def limit_weeks(df):
        week_columns = [col for col in df.columns if col not in ['Row Labels', 'Grand Total']]
        sorted_week_columns = sorted(week_columns)
        limited_columns = sorted_week_columns[-49:]   
        columns_to_keep = ['Row Labels'] + limited_columns + ['Grand Total'] 
        valid_columns = [col for col in columns_to_keep if col in df.columns]   
        filtered_df = df[valid_columns]    
        return filtered_df

    def process_excel(path):
        df = pd.read_excel(path)
        row_labels_index = df[df.eq("Row Labels").any(axis=1)].index[0]
        df.columns = df.iloc[row_labels_index]
        df = df.iloc[row_labels_index + 1:, :].reset_index(drop=True)
        df = df.dropna(how='all', axis=0).dropna(how='all', axis=1)
        df['Row Labels'] = df['Row Labels'].replace([0, "0"], "LineX1") # as there is one row with 0 label in some files
        df = df.fillna(0)
        df.columns = [col.replace('Sum of ', '') for col in df.columns]
        df['Row Labels'] = df['Row Labels'].replace(["...", "'"], "LineX1") # as there is one row with 0 label in some files
        df['Row Labels'] = df['Row Labels'].replace([0, "0"], "LineX2") # as there is one row with Nan (so doing it after filling Nans) in some files
        df = df[~df['Row Labels'].str.contains(r'X', regex=True)]
        df = df[~df['Row Labels'].str.contains(r'^CW', regex=True)]
        df = df[~df['Row Labels'].str.contains(r'blank', regex=True)]
        week_columns = [col for col in df.columns if col != 'Row Labels']
        sorted_week_columns = sorted(week_columns, key=lambda x: int(x))
        df = df[['Row Labels'] + sorted_week_columns]
        df['Grand Total'] = df.loc[:, df.columns != 'Row Labels'].sum(axis=1)
        if len(df.index) == 9 and "Grand Total" not in df['Row Labels']:
            df = addGrandTotal(df)
        df = renameColumns(df)
        df = limit_weeks(df)
        df = df.drop(columns=['Grand Total'])
        df = df[df['Row Labels'] != 'Grand Total']
        return df

    merged_df = pd.DataFrame()
    for file in os.listdir(path):
        if file.endswith(".xlsx"):
            file_path = os.path.join(path, file)
            df = process_excel(file_path)
            if merged_df.empty:
                merged_df = df
            else:
                new_columns = [col for col in df.columns if col not in merged_df.columns and col != 'Row Labels']
                df_new_weeks = df[['Row Labels'] + new_columns]
                merged_df = pd.merge(merged_df, df_new_weeks, on='Row Labels', how='outer')

    return merged_df

# Define paths
sales_data_path = 'E:/Fiverr/1st/Data/'
demand_data_path = 'E:/Fiverr/1st/Data/Demand update every week/'

# Process datasets
merged_df_sales = preprocess_sales_data(sales_data_path)
merged_df_demand = preprocess_demand_data(demand_data_path)

# Align them by week and year (assuming both have the same 'Row Labels' values)
aligned_df = pd.merge(merged_df_sales, merged_df_demand, on='Row Labels', how='inner', suffixes=('_sales', '_demand'))

aligned_df

df = aligned_df.copy()

# Transpose the data to have weeks as rows and lines as columns
df = df.set_index('Row Labels').transpose()
df = df[~df.index.str.contains("2025")]
df['year'] = df.index.map(lambda entry: int(entry[0:4]))
df['week'] = df.index.map(lambda entry: int(entry[5:7]))

df

Row Labels,Line 1-2,Line 2,Line 3,Line 4,Line 5,Line 6,Line 7,SGB,year,week
2020-01,2624.0,8522.0,12831.0,19931.0,18144.0,5142.0,10410.0,10574.0,2020,1
2020-02,207.0,16235.0,14605.0,10160.0,9754.0,15605.0,14730.0,8362.0,2020,2
2020-03,2343.0,6697.0,5274.0,19700.0,9742.0,7238.0,18360.0,19078.0,2020,3
2020-04,2664.0,12221.0,17645.0,18430.0,8607.0,14127.0,17242.0,12191.0,2020,4
2020-05,647.0,13086.0,5380.0,14900.0,17574.0,8318.0,12578.0,9762.0,2020,5
...,...,...,...,...,...,...,...,...,...,...
2024-44,0.0,2550.0,0.0,1780.0,2010.0,422.0,1788.0,1800.0,2024,44
2024-47,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2040.0,2024,47
2024-48,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1500.0,2024,48
2024-50,0.0,0.0,0.0,1600.0,2420.0,22.0,0.0,66.0,2024,50


# Feature Engineering for better results

In [17]:
# Create lag features
lag_features = 1  # You can adjust the number of lags if needed

for column in df.columns[:-2]:  # Exclude 'year' and 'week' columns
    for lag in range(1, lag_features + 1):
        df[f'{column}_lag{lag}'] = df[column].shift(lag)
df

Row Labels,Line 1-2,Line 2,Line 3,Line 4,Line 5,Line 6,Line 7,SGB,year,week,Line 1-2_lag1,Line 2_lag1,Line 3_lag1,Line 4_lag1,Line 5_lag1,Line 6_lag1,Line 7_lag1,SGB_lag1
2020-01,2624.0,8522.0,12831.0,19931.0,18144.0,5142.0,10410.0,10574.0,2020,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-02,207.0,16235.0,14605.0,10160.0,9754.0,15605.0,14730.0,8362.0,2020,2,2624.0,8522.0,12831.0,19931.0,18144.0,5142.0,10410.0,10574.0
2020-03,2343.0,6697.0,5274.0,19700.0,9742.0,7238.0,18360.0,19078.0,2020,3,207.0,16235.0,14605.0,10160.0,9754.0,15605.0,14730.0,8362.0
2020-04,2664.0,12221.0,17645.0,18430.0,8607.0,14127.0,17242.0,12191.0,2020,4,2343.0,6697.0,5274.0,19700.0,9742.0,7238.0,18360.0,19078.0
2020-05,647.0,13086.0,5380.0,14900.0,17574.0,8318.0,12578.0,9762.0,2020,5,2664.0,12221.0,17645.0,18430.0,8607.0,14127.0,17242.0,12191.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-44,0.0,2550.0,0.0,1780.0,2010.0,422.0,1788.0,1800.0,2024,44,0.0,0.0,0.0,852.0,0.0,0.0,0.0,0.0
2024-47,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2040.0,2024,47,0.0,2550.0,0.0,1780.0,2010.0,422.0,1788.0,1800.0
2024-48,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1500.0,2024,48,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2040.0
2024-50,0.0,0.0,0.0,1600.0,2420.0,22.0,0.0,66.0,2024,50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1500.0


In [18]:
# Create moving average features
window_size = 3  # Moving average window size, can be adjusted

for column in df.columns[:-2]:  # Exclude 'year' and 'week' columns
    df[f'{column}_ma{window_size}'] = df[column].rolling(window=window_size).mean()
df

Row Labels,Line 1-2,Line 2,Line 3,Line 4,Line 5,Line 6,Line 7,SGB,year,week,...,Line 7_ma3,SGB_ma3,year_ma3,week_ma3,Line 1-2_lag1_ma3,Line 2_lag1_ma3,Line 3_lag1_ma3,Line 4_lag1_ma3,Line 5_lag1_ma3,Line 6_lag1_ma3
2020-01,2624.0,8522.0,12831.0,19931.0,18144.0,5142.0,10410.0,10574.0,2020,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-02,207.0,16235.0,14605.0,10160.0,9754.0,15605.0,14730.0,8362.0,2020,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-03,2343.0,6697.0,5274.0,19700.0,9742.0,7238.0,18360.0,19078.0,2020,3,...,14500.000000,12671.333333,2020.0,2.000000,NaN,NaN,NaN,NaN,NaN,NaN
2020-04,2664.0,12221.0,17645.0,18430.0,8607.0,14127.0,17242.0,12191.0,2020,4,...,16777.333333,13210.333333,2020.0,3.000000,1724.666667,10484.666667,10903.333333,16597.000000,12546.666667,9328.333333
2020-05,647.0,13086.0,5380.0,14900.0,17574.0,8318.0,12578.0,9762.0,2020,5,...,16060.000000,13677.000000,2020.0,4.000000,1738.000000,11717.666667,12508.000000,16096.666667,9367.666667,12323.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-44,0.0,2550.0,0.0,1780.0,2010.0,422.0,1788.0,1800.0,2024,44,...,1196.333333,2607.333333,2024.0,46.333333,0.000000,0.000000,0.000000,1472.000000,1216.666667,7.333333
2024-47,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2040.0,2024,47,...,596.000000,1280.000000,2024.0,46.666667,0.000000,850.000000,0.000000,1471.333333,1403.333333,148.000000
2024-48,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1500.0,2024,48,...,596.000000,1780.000000,2024.0,46.333333,0.000000,850.000000,0.000000,877.333333,670.000000,140.666667
2024-50,0.0,0.0,0.0,1600.0,2420.0,22.0,0.0,66.0,2024,50,...,0.000000,1202.000000,2024.0,48.333333,0.000000,850.000000,0.000000,593.333333,670.000000,140.666667


In [19]:
import numpy as np

# Encode week of the year as cyclical features
df['week_sin'] = np.sin(2 * np.pi * df['week'] / 52)
df['week_cos'] = np.cos(2 * np.pi * df['week'] / 52)
df

Row Labels,Line 1-2,Line 2,Line 3,Line 4,Line 5,Line 6,Line 7,SGB,year,week,...,year_ma3,week_ma3,Line 1-2_lag1_ma3,Line 2_lag1_ma3,Line 3_lag1_ma3,Line 4_lag1_ma3,Line 5_lag1_ma3,Line 6_lag1_ma3,week_sin,week_cos
2020-01,2624.0,8522.0,12831.0,19931.0,18144.0,5142.0,10410.0,10574.0,2020,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.120537,0.992709
2020-02,207.0,16235.0,14605.0,10160.0,9754.0,15605.0,14730.0,8362.0,2020,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.239316,0.970942
2020-03,2343.0,6697.0,5274.0,19700.0,9742.0,7238.0,18360.0,19078.0,2020,3,...,2020.0,2.000000,NaN,NaN,NaN,NaN,NaN,NaN,0.354605,0.935016
2020-04,2664.0,12221.0,17645.0,18430.0,8607.0,14127.0,17242.0,12191.0,2020,4,...,2020.0,3.000000,1724.666667,10484.666667,10903.333333,16597.000000,12546.666667,9328.333333,0.464723,0.885456
2020-05,647.0,13086.0,5380.0,14900.0,17574.0,8318.0,12578.0,9762.0,2020,5,...,2020.0,4.000000,1738.000000,11717.666667,12508.000000,16096.666667,9367.666667,12323.333333,0.568065,0.822984
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-44,0.0,2550.0,0.0,1780.0,2010.0,422.0,1788.0,1800.0,2024,44,...,2024.0,46.333333,0.000000,0.000000,0.000000,1472.000000,1216.666667,7.333333,-0.822984,0.568065
2024-47,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2040.0,2024,47,...,2024.0,46.666667,0.000000,850.000000,0.000000,1471.333333,1403.333333,148.000000,-0.568065,0.822984
2024-48,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1500.0,2024,48,...,2024.0,46.333333,0.000000,850.000000,0.000000,877.333333,670.000000,140.666667,-0.464723,0.885456
2024-50,0.0,0.0,0.0,1600.0,2420.0,22.0,0.0,66.0,2024,50,...,2024.0,48.333333,0.000000,850.000000,0.000000,593.333333,670.000000,140.666667,-0.239316,0.970942


In [20]:
# Interaction term between year and week
df['year_week_interaction'] = df['year'] * df['week']
df

Row Labels,Line 1-2,Line 2,Line 3,Line 4,Line 5,Line 6,Line 7,SGB,year,week,...,week_ma3,Line 1-2_lag1_ma3,Line 2_lag1_ma3,Line 3_lag1_ma3,Line 4_lag1_ma3,Line 5_lag1_ma3,Line 6_lag1_ma3,week_sin,week_cos,year_week_interaction
2020-01,2624.0,8522.0,12831.0,19931.0,18144.0,5142.0,10410.0,10574.0,2020,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.120537,0.992709,2020
2020-02,207.0,16235.0,14605.0,10160.0,9754.0,15605.0,14730.0,8362.0,2020,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.239316,0.970942,4040
2020-03,2343.0,6697.0,5274.0,19700.0,9742.0,7238.0,18360.0,19078.0,2020,3,...,2.000000,NaN,NaN,NaN,NaN,NaN,NaN,0.354605,0.935016,6060
2020-04,2664.0,12221.0,17645.0,18430.0,8607.0,14127.0,17242.0,12191.0,2020,4,...,3.000000,1724.666667,10484.666667,10903.333333,16597.000000,12546.666667,9328.333333,0.464723,0.885456,8080
2020-05,647.0,13086.0,5380.0,14900.0,17574.0,8318.0,12578.0,9762.0,2020,5,...,4.000000,1738.000000,11717.666667,12508.000000,16096.666667,9367.666667,12323.333333,0.568065,0.822984,10100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-44,0.0,2550.0,0.0,1780.0,2010.0,422.0,1788.0,1800.0,2024,44,...,46.333333,0.000000,0.000000,0.000000,1472.000000,1216.666667,7.333333,-0.822984,0.568065,89056
2024-47,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2040.0,2024,47,...,46.666667,0.000000,850.000000,0.000000,1471.333333,1403.333333,148.000000,-0.568065,0.822984,95128
2024-48,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1500.0,2024,48,...,46.333333,0.000000,850.000000,0.000000,877.333333,670.000000,140.666667,-0.464723,0.885456,97152
2024-50,0.0,0.0,0.0,1600.0,2420.0,22.0,0.0,66.0,2024,50,...,48.333333,0.000000,850.000000,0.000000,593.333333,670.000000,140.666667,-0.239316,0.970942,101200


In [21]:
# Drop rows with NaN values generated from lag features and moving averages
df = df.dropna().reset_index(drop=True)
df

Row Labels,Line 1-2,Line 2,Line 3,Line 4,Line 5,Line 6,Line 7,SGB,year,week,...,week_ma3,Line 1-2_lag1_ma3,Line 2_lag1_ma3,Line 3_lag1_ma3,Line 4_lag1_ma3,Line 5_lag1_ma3,Line 6_lag1_ma3,week_sin,week_cos,year_week_interaction
0,2664.0,12221.0,17645.0,18430.0,8607.0,14127.0,17242.0,12191.0,2020,4,...,3.000000,1724.666667,10484.666667,10903.333333,16597.000000,12546.666667,9328.333333,0.464723,0.885456,8080
1,647.0,13086.0,5380.0,14900.0,17574.0,8318.0,12578.0,9762.0,2020,5,...,4.000000,1738.000000,11717.666667,12508.000000,16096.666667,9367.666667,12323.333333,0.568065,0.822984,10100
2,2003.0,14779.0,5642.0,16231.0,6501.0,17922.0,16661.0,13020.0,2020,6,...,5.000000,1884.666667,10668.000000,9433.000000,17676.666667,11974.333333,9894.333333,0.663123,0.748511,12120
3,2472.0,17347.0,5848.0,5023.0,8490.0,9567.0,14699.0,17361.0,2020,7,...,6.000000,1771.333333,13362.000000,9555.666667,16520.333333,10894.000000,13455.666667,0.748511,0.663123,14140
4,2931.0,8344.0,12703.0,5140.0,6317.0,9722.0,12494.0,7321.0,2020,8,...,7.000000,1707.333333,15070.666667,5623.333333,12051.333333,10855.000000,11935.666667,0.822984,0.568065,16160
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331,0.0,2550.0,0.0,1780.0,2010.0,422.0,1788.0,1800.0,2024,44,...,46.333333,0.000000,0.000000,0.000000,1472.000000,1216.666667,7.333333,-0.822984,0.568065,89056
332,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2040.0,2024,47,...,46.666667,0.000000,850.000000,0.000000,1471.333333,1403.333333,148.000000,-0.568065,0.822984,95128
333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1500.0,2024,48,...,46.333333,0.000000,850.000000,0.000000,877.333333,670.000000,140.666667,-0.464723,0.885456,97152
334,0.0,0.0,0.0,1600.0,2420.0,22.0,0.0,66.0,2024,50,...,48.333333,0.000000,850.000000,0.000000,593.333333,670.000000,140.666667,-0.239316,0.970942,101200


# Splitting Dataset

In [22]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

target_columns = ['Line 1-2', 'Line 2', 'Line 3', 'Line 4', 'Line 5', 'Line 6', 'Line 7', 'SGB'] 
X = df.drop(columns=target_columns)
y = df[target_columns]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Training Model

In [23]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

# Define the parameter grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}

# Initialize the Random Forest model
rf = RandomForestRegressor(random_state=42)

# Initialize the Grid Search with cross-validation
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, n_jobs=-1, scoring='neg_mean_squared_error')

# Fit the model
grid_search.fit(X_train, y_train)

# Get the best parameters
best_params = grid_search.best_params_
print(f'Best parameters found: {best_params}')

# Re-train the Random Forest with the best parameters
best_rf = grid_search.best_estimator_

Best parameters found: {'max_depth': 20, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 300}


In [42]:
from sklearn.feature_selection import SelectFromModel

# Use the best estimator from Grid Search to select important features
selector = SelectFromModel(best_rf, threshold='mean')
selector.fit(X_train, y_train)

# Transform the datasets
X_train_selected = selector.transform(X_train)
X_test_selected = selector.transform(X_test)

# Re-train the model using selected features
best_rf.fit(X_train_selected, y_train)
y_pred_selected = best_rf.predict(X_test_selected)

# Evaluate the model performance
mse_selected = mean_squared_error(y_test, y_pred_selected)
print(f'Mean Squared Error after feature selection: {mse_selected}')

Mean Squared Error after feature selection: 79.113573


In [43]:
from sklearn.model_selection import cross_val_score

# Cross-validation on the model with selected features
cv_scores = cross_val_score(best_rf, X_train_selected, y_train, cv=5, scoring='neg_mean_squared_error')
mean_cv_score = -cv_scores.mean()
print(f'Mean Cross-Validation MSE: {mean_cv_score}')

Mean Cross-Validation MSE: 72.473922


In [44]:
# Final prediction using the best model
final_predictions = best_rf.predict(X_test_selected)

# Final MSE
final_mse = mean_squared_error(y_test, final_predictions)
print(f'Final Mean Squared Error: {final_mse}')

Final Mean Squared Error: 78.2939374


# Getting Prediction from Model

In [126]:
# Extract the rows for week 28 and week 47 of 2024
df_2024 = df[(df['year'] == 2023) ]

X_new = df_2024.drop(columns=target_columns)

X_new_selected = selector.transform(X_new)

# Make predictions for week 28 and week 47
predictions_2024 = best_rf.predict(X_new_selected)

# Create a DataFrame to display the predictions alongside the week
prediction_df = pd.DataFrame(predictions_2024, columns=target_columns, index=df_2024['week'])
prediction_df

,Line 1-2,Line 2,Line 3,Line 4,Line 5,Line 6,Line 7,SGB
week,,,,,,,,
1,1065.176534,14099.050742,11261.101264,9307.059381,7818.351628,6208.844521,4277.647731,20332.537362
2,1887.334302,24974.727491,27827.715139,16219.671410,13302.530925,9354.942103,1479.052940,21618.868266
3,1599.110706,22495.221437,23338.296953,14452.086124,11140.734567,8296.825671,1533.759925,23251.598995
4,1498.550185,30841.416642,32660.705049,19374.945780,14585.220388,10687.014650,1913.682743,30501.280629
5,1582.546423,21246.914192,23780.037847,14289.541467,11382.080800,8643.606911,1231.490914,21292.993911
...,...,...,...,...,...,...,...,...
11,2267.885376,41202.603001,38747.975333,26005.397714,16440.255735,13769.211887,2429.093657,34880.551351
12,1091.098133,17456.769944,16882.994705,11048.117718,8184.984847,5458.911438,1331.410562,16165.795807
13,1577.401707,19958.244298,23791.082833,14903.312512,10517.088290,8061.041174,1085.658915,16493.793146


# Saving the trained model for Dashboards

In [27]:
import joblib

# Save the trained model and the selector to a file
model_filename = 'E:/Fiverr/1st/best_random_forest_model.pkl'
selector_filename = 'E:/Fiverr/1st/selector.pkl'

joblib.dump(best_rf, model_filename)
joblib.dump(selector, selector_filename)

print(f'Model saved as {model_filename} and selector saved as {selector_filename}')


Model saved as E:/Fiverr/1st/best_random_forest_model.pkl and selector saved as E:/Fiverr/1st/selector.pkl


# Loading saved model for testing

In [39]:
import joblib

model_filename = 'E:/Fiverr/1st/best_random_forest_model.pkl'
selector_filename = 'E:/Fiverr/1st/selector.pkl'

loaded_model = joblib.load(model_filename)
selector = joblib.load(selector_filename)

df_2024 = df[(df['year'] == 2024) ]

X_new = df_2024.drop(columns=target_columns)

X_new_selected = selector.transform(X_new)

predictions = loaded_model.predict(X_new_selected)

data_normalized = predictions.astype(float)

column_names = ['Line 1-2', 'Line 2', 'Line 3', 'Line 4', 
                'Line 5', 'Line 6', 'Line 7', 'SGB']
prediction_df = pd.DataFrame(data_normalized, columns=column_names)

prediction_df = prediction_df.tail(51).reset_index(drop=True)
prediction_df['week'] = 1 + prediction_df.index
prediction_df

,Line 1-2,Line 2,Line 3,Line 4,Line 5,Line 6,Line 7,SGB,week
0,967.356362,11907.500425,10516.625509,8371.150765,5575.256853,3486.055752,1438.153967,11521.077209,1
1,1732.860062,16941.896967,18829.345073,12883.479778,10163.459467,6699.656009,1744.146649,14971.738819,2
2,1127.966114,29940.498201,31892.450839,26109.875772,12440.695488,13066.694213,1569.796813,31615.138487,3
3,1229.618580,29887.084701,31313.217083,21161.073123,11492.095553,12209.782224,1550.239319,27310.132701,4
4,1256.635251,26702.261071,29878.844890,19490.245526,12780.266256,11109.437336,1799.270549,26164.290919,5
5,1040.205405,21294.218353,20866.072923,11337.716437,6972.644300,9078.404926,1848.137862,18532.030610,6
6,1044.588112,18414.315302,18326.005193,10103.128974,6952.111951,7405.371604,1072.174151,18000.577894,7
7,794.773059,17795.583558,17160.369609,7860.337837,5158.188875,7295.956753,1012.003909,17098.847861,8
8,848.910866,16048.053863,15560.028474,7189.082880,4793.668216,6663.152135,988.058367,15755.605060,9
9,999.705472,17939.561778,17147.850692,8022.091736,5484.848984,7333.910063,1147.145458,14413.005033,10
